## Accessing information about LOMRs

In [ ]:
from arcgis.features import FeatureLayer
from arcgis.features import FeatureLayerCollection
from arcgis.geometry import filters
import datetime

Convert relevant endpoint URLs into ESRI FeatureLayer objects

In [ ]:
city_lims_url = "https://maps.bouldercolorado.gov/arcgis/rest/services/plan/CityLimits/MapServer/0"
city = FeatureLayer(city_lims_url)

nfhl_url = "https://hazards.fema.gov/gis/nfhl/rest/services/public/NFHL/MapServer"
nfhl = FeatureLayerCollection(nfhl_url)
lomr = nfhl.layers[1]

Define the spatial boundaries to test within

In [ ]:
city_ext = city.properties.extent
geom_filter = filters.contains(city_ext)

Perform the query against the LOMR service

In [ ]:
# return as a pandas spatial df
sr = 2876
boulder_lomrs = lomr.query(geometry_filter=geom_filter, out_sr=sr, as_df=True)

Find the most recent update to NFHL floodplains in Boulder

In [ ]:
# returns as a timestamp
most_recent_lomr = boulder_lomrs.EFF_DATE.max()
most_recent_lomr

## Accessing information about LOMAs

In [ ]:
loma = nfhl.layers[2]

Based on a little bit of data exploration, we know that the City of Boulder is specifically called out in the COMMUNITY field. Let's see how well the counts match based on a "WHERE" style SQL query vs. a spatial query

In [ ]:
where_statement = "COMMUNITY LIKE '%BOULDER, CITY OF%'"

sql_query = loma.query(where=where_statement, out_sr=sr, as_df=True)
spatial_query = loma.query(geometry_filter=geom_filter, out_sr=sr, as_df=True)

# compare number of records
len(spatial_query) == len(sql_query)